requires the installation of the docker images for rf5 and whisper and the starting of the docker container

docker run -d -p 9000:9000 -e ASR_MODEL=medium thoppe/openai-whisper-asr-webservice-predownload-medium:12-08-2022
docker run -d -p 9001:9000 thoppe/rf5_speaker_embedding_webservice

In [46]:
import os
import requests
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering
import pandas as pd

In [2]:
WHISPER_PORT = 9002
whisper_url = f"http://localhost:{WHISPER_PORT}/asr"

In [9]:
file_audio = 'AndreIguodala_ESPN'
f_audio = f"C:/Users/qtw4/OneDrive - CDC/RANDS/whisper/whisper pipeline/raw/{file_audio}.mp3"

In [7]:
with open(f_audio, "rb") as FIN:
    raw_audio = FIN.read()

r = requests.post(whisper_url, files={"audio_file": raw_audio})
assert r.ok

js = r.json()
json_string = json.dumps(js)
print(js["text"])

 I want Iguodala. That's right. And I know what that sounds like. And it's not fair to Steph because so much defensive attention is paid to him. And I'm not saying Steph's not a better shooter. He's a way better shooter. Iguodala's got ice water in his veins. My money is on number 30.


In [11]:
json_string = json.dumps(js)
with open(f'{file_audio}.json', 'w') as outfile:
    outfile.write(json_string)

In [18]:
file_json = open(f'{file_audio}.json',)
data = json.load(file_json)
EMBED_PORT = 9001
embed_url = f"http://localhost:{EMBED_PORT}/embed"

In [19]:
data

{'text': " I want Iguodala. That's right. And I know what that sounds like. And it's not fair to Steph because so much defensive attention is paid to him. And I'm not saying Steph's not a better shooter. He's a way better shooter. Iguodala's got ice water in his veins. My money is on number 30.",
 'segments': [{'id': 0,
   'seek': 0,
   'start': 0.0,
   'end': 3.0,
   'text': " I want Iguodala. That's right.",
   'tokens': [286, 528, 286, 2794, 378, 5159, 13, 663, 311, 558, 13],
   'temperature': 0.0,
   'avg_logprob': -0.20760823367686754,
   'compression_ratio': 1.518716577540107,
   'no_speech_prob': 0.05235247686505318},
  {'id': 1,
   'seek': 0,
   'start': 3.0,
   'end': 4.5,
   'text': ' And I know what that sounds like.',
   'tokens': [400, 286, 458, 437, 300, 3263, 411, 13],
   'temperature': 0.0,
   'avg_logprob': -0.20760823367686754,
   'compression_ratio': 1.518716577540107,
   'no_speech_prob': 0.05235247686505318},
  {'id': 2,
   'seek': 0,
   'start': 4.5,
   'end': 8.2

In [24]:
#open file as raw bites

with open(f'{file_audio}.json', "rb") as FIN:
    whisper_content = FIN.read()

In [26]:
r = requests.post(embed_url, files={"audio_file": raw_audio, "whisper_file": whisper_content})
assert r.ok

array([0, 0, 0, 0, 0, 0, 1])

In [49]:
vecs = r.json()
array = np.array(vecs)
#segments = pd.DataFrame(js['segments'])
kmeans = KMeans(n_clusters=2, random_state=0).fit(array)
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 1])

In [27]:
from sklearn.cluster import DBSCAN

In [50]:
dbscan_cluster = DBSCAN(eps=1).fit(array)
dbscan_cluster.labels_

array([ 0,  0,  0,  0,  0,  0, -1], dtype=int64)

In [ ]:
#assigns to cluster - but can throw out based on the audio
#docs - says throws out based on audio, doesn't find speaker until end, so it sees it as one speaker

In [51]:
clustering = SpectralClustering(n_clusters=2, random_state=0).fit(array)
clustering.labels_

array([1, 1, 1, 1, 1, 1, 0])

In [ ]:
segments['speaker'] = kmeans.labels_

In [ ]:
sim = np.round(cosine_similarity(vecs), 2)
print(sim)

In [6]:
# Get a list of all files in the RAW folder
raw_files = os.listdir("raw")

# Iterate over the files in the RAW folder
for file in raw_files:
    # Load the file from the RAW folder
    f_audio = os.path.join("raw", file)
    
    with open(f_audio, "rb") as FIN:
        raw_audio = FIN.read()
    # Run the file through the model
    r = requests.post(whisper_url, files={"audio_file": raw_audio})
    assert r.ok
   
    js = r.json()
    file_name = f'whisper_transcription_:{file}'
    # output 
    with open("json_results"/f'whisper_transcription_:{file}', 'w') as outfile:
        json.dump(js, outfile)
    print('Completed Transcription: ', file)

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [ ]:
WHISPER_PORT = 9000
EMBED_PORT = 9001

whisper_url = f"http://localhost:{WHISPER_PORT}/asr"
embed_url = f"http://localhost:{EMBED_PORT}/embed"

f_audio = "espn_first_take.mp3"

with open(f_audio, "rb") as FIN:
    raw_audio = FIN.read()